<a href="https://colab.research.google.com/github/mhpbreugem/WP1/blob/main/BBP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
using Distributions, Random, NLsolve, LinearAlgebra, Printf, Optim, BenchmarkTools

In [41]:
# Chapter 0: Parameters

# Economic parameters
β = 0.95
γ = 4.0
e0 = 3.0
EΠ = 1.0
Π0 = 1.0
θXbar = 1.0
θYbar = 1.0
μθX = 0.0
σθX = 0.2
μθY = 0.0
σθY = 0.1
Nσπ = 2
σΠ = 0.4
ρ = 0
W0 = e0

ΞЮ = [repeat([x], 1) for x in range(0.1, stop=1.0, length=10)]

# Grid parameters
Nσπ = 17
NNσπ = 1
Nπ = 2 * Nσπ * NNσπ + 1  # number of grid points for payoff

NσΠ = 2
NNσΠ = 1
NΠ = 2 * NσΠ * NNσΠ + 1  # number of grid points for payoff

NθX = 5
NθY = 5
NσθX = 2
NσθY = 2

# Derived midpoints
μπi = (Nπ + 1) ÷ 2
μΠi = (NΠ + 1) ÷ 2
μθXi = (NθX + 1) ÷ 2

# Total number of states
NS = Nπ
NN = NθX * NθY * NΠ

125

In [44]:
# Chapter 1: Grid Construction

# Conjectured Payoffs
ππ = collect(range(EΠ - Nσπ*σΠ, EΠ + Nσπ*σΠ, length=Nπ))
πω = pdf.(Normal(EΠ, σΠ), ππ)
πΩ = πω ./ sum(πω)

# Payoff Parameters
Δ1 = Π0
Δ2 = ππ

# True Payoffs
ΞΠ = EΠ .+ σΠ .* range(-NσΠ, NσΠ, length=NΠ)
ππ_to_index = Dict(round(ππ[πi], digits=8) => πi for πi in 1:Nπ)
ΞΠi = [ππ_to_index[round(ΞΠ[Πi], digits=8)] for Πi in 1:NΠ]

# Signals
S=ππ

@inline function ψϵω(Ю, ϵ)
    σϵ = 1 / sqrt(Ю)
    pdf(Normal(0.0, σϵ), ϵ)
end

function φSΩπ(Ю)
    σϵ = 1 / sqrt(Ю[1])
    SΩ = [pdf(Normal(π, σϵ), s) for π in ππ, s in ππ]
    SΩnorm = SΩ ./ sum(SΩ, dims=2)
    return SΩnorm
end

# Noise Trader Demand
ΞθX = μθX .+ σθX .* range(-NσθX, NσθX, length=NθX)
ΞθY = μθY .+ σθY .* range(-NσθY, NσθY, length=NθY)

const Σθ = [σθX^2 σθX*σθY*ρ; σθX*σθY*ρ σθY^2]
const dist = MvNormal([μθX, μθY], Σθ)
θXYωω(x, y) = pdf(dist, [x, y])

θXYωω (generic function with 1 method)

In [45]:
# Chapter 2: Learning

# Full learning (private + public information)
function φΞΩI(Ю, XΣ, YΣ)
    ϵ = S .- ππ'
    ψϵ_values = ψϵω.(Ю, ϵ)
    θXY_values = θXYωω.(θXbar .- XΣ, θYbar .- YΣ)

    ΞΩI = πΩ .* ψϵ_values .* θXY_values
    sum_ΞΩI = sum(ΞΩI, dims=1)
    ΞΩI .= ΞΩI ./ sum_ΞΩI

    return ΞΩI
end

φΞΩI (generic function with 1 method)

In [23]:
@btime φΞΩI([0.1], fill(1,NS), fill(1,NS))

  17.292 μs (165 allocations: 39.92 KiB)


35×35 Matrix{Float64}:
 6.87196e-63  5.25724e-63  4.02093e-63  …  8.72286e-67  6.61798e-67
 9.9877e-56   7.76411e-56  6.03404e-56     2.14957e-59  1.65717e-59
 5.25541e-49  4.15128e-49  3.27829e-49     1.91779e-52  1.50232e-52
 1.00116e-42  8.03579e-43  6.44826e-43     6.19449e-46  4.93081e-46
 6.90494e-37  5.63161e-37  4.59193e-37     7.24383e-40  5.85908e-40
 1.72414e-31  1.42887e-31  1.18387e-31  …  3.06681e-34  2.52056e-34
 1.55862e-26  1.31253e-26  1.10502e-26     4.70071e-29  3.92575e-29
 5.10112e-22  4.365e-22    3.73416e-22     2.60853e-24  2.21362e-24
 6.04433e-18  5.25551e-18  4.56849e-18     5.24066e-20  4.519e-20
 2.59291e-14  2.29088e-14  2.02353e-14     3.81182e-16  3.33993e-16
 4.02701e-11  3.61533e-11  3.24491e-11  …  1.00377e-12  8.93695e-13
 2.26431e-8   2.06561e-8   1.88388e-8      9.56965e-10  8.65762e-10
 4.60941e-6   4.27275e-6   3.95968e-6      3.30304e-7   3.03644e-7
 ⋮                                      ⋱               
 8.65762e-10  9.56965e-10  1.05751e-9  

1×35 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [54]:
# === Cached Struct and Optimized Function Implementation === #

mutable struct CachedΞΩI
    cached_ϵ::Matrix{Float64}
    cached_ψϵ_values::Matrix{Float64}
    cached_θXY_values::Matrix{Float64}
    cached_ΞΩI::Matrix{Float64}
    cached_sum_ΞΩI::Matrix{Float64}
end

function CachedΞΩI(S, ππ)
    NS, Nπ = length(S), length(ππ)
    CachedΞΩI(
        zeros(NS, Nπ),
        zeros(NS, Nπ),
        zeros(NS, Nπ),
        zeros(NS, Nπ),
        zeros(1, Nπ)
    )
end

# Marking functions with @inline for efficiency
@inline function ψϵω(Ю, ϵ)
    σϵ = 1 / sqrt(Ю)
    pdf(Normal(0.0, σϵ), ϵ)
end

const θ_dist = MvNormal([μθX, μθY], [σθX^2 σθX*σθY*ρ; σθX*σθY*ρ σθY^2])

@inline function θXYωω(x, y)
    pdf(θ_dist, [x, y])
end

# Optimized cached function using struct-based caching
function φΞΩI!(cache::CachedΞΩI, Ю, XΣ, YΣ)
    @. cache.cached_ϵ = S - ππ'

    σϵ = 1 / sqrt(Ю)
    @inbounds for i in eachindex(cache.cached_ϵ)
        cache.cached_ψϵ_values[i] = ψϵω(Ю, cache.cached_ϵ[i])
    end

    @inbounds for i in eachindex(XΣ)
        val = θXYωω(θXbar - XΣ[i], θYbar - YΣ[i])
        cache.cached_θXY_values[i, :] .= val
    end

    @. cache.cached_ΞΩI = πΩ * cache.cached_ψϵ_values * cache.cached_θXY_values

    cache.cached_sum_ΞΩI .= sum(cache.cached_ΞΩI, dims=1)
    cache.cached_ΞΩI ./= cache.cached_sum_ΞΩI

    return cache.cached_ΞΩI
end

# Initialize cache once
cache = CachedΞΩI(S, ππ)

# Benchmark clearly to confirm efficiency
using BenchmarkTools
@btime φΞΩI!($cache, 0.1, fill(1,NS), fill(1,NS))


  24.927 μs (358 allocations: 14.25 KiB)


35×35 Matrix{Float64}:
 6.87196e-63  5.25724e-63  4.02093e-63  …  8.72286e-67  6.61798e-67
 9.9877e-56   7.76411e-56  6.03404e-56     2.14957e-59  1.65717e-59
 5.25541e-49  4.15128e-49  3.27829e-49     1.91779e-52  1.50232e-52
 1.00116e-42  8.03579e-43  6.44826e-43     6.19449e-46  4.93081e-46
 6.90494e-37  5.63161e-37  4.59193e-37     7.24383e-40  5.85908e-40
 1.72414e-31  1.42887e-31  1.18387e-31  …  3.06681e-34  2.52056e-34
 1.55862e-26  1.31253e-26  1.10502e-26     4.70071e-29  3.92575e-29
 5.10112e-22  4.365e-22    3.73416e-22     2.60853e-24  2.21362e-24
 6.04433e-18  5.25551e-18  4.56849e-18     5.24066e-20  4.519e-20
 2.59291e-14  2.29088e-14  2.02353e-14     3.81182e-16  3.33993e-16
 4.02701e-11  3.61533e-11  3.24491e-11  …  1.00377e-12  8.93695e-13
 2.26431e-8   2.06561e-8   1.88388e-8      9.56965e-10  8.65762e-10
 4.60941e-6   4.27275e-6   3.95968e-6      3.30304e-7   3.03644e-7
 ⋮                                      ⋱               
 8.65762e-10  9.56965e-10  1.05751e-9  

In [37]:
@btime φΞΩI_cached_final(0.1, fill(1,NS), fill(1,NS), $cached_ϵ, $cached_ψϵ_values, $cached_θXY_values, $cached_ΞΩI, $cached_sum_ΞΩI)


  21.306 μs (358 allocations: 14.25 KiB)


35×35 Matrix{Float64}:
 6.87196e-63  5.25724e-63  4.02093e-63  …  8.72286e-67  6.61798e-67
 9.9877e-56   7.76411e-56  6.03404e-56     2.14957e-59  1.65717e-59
 5.25541e-49  4.15128e-49  3.27829e-49     1.91779e-52  1.50232e-52
 1.00116e-42  8.03579e-43  6.44826e-43     6.19449e-46  4.93081e-46
 6.90494e-37  5.63161e-37  4.59193e-37     7.24383e-40  5.85908e-40
 1.72414e-31  1.42887e-31  1.18387e-31  …  3.06681e-34  2.52056e-34
 1.55862e-26  1.31253e-26  1.10502e-26     4.70071e-29  3.92575e-29
 5.10112e-22  4.365e-22    3.73416e-22     2.60853e-24  2.21362e-24
 6.04433e-18  5.25551e-18  4.56849e-18     5.24066e-20  4.519e-20
 2.59291e-14  2.29088e-14  2.02353e-14     3.81182e-16  3.33993e-16
 4.02701e-11  3.61533e-11  3.24491e-11  …  1.00377e-12  8.93695e-13
 2.26431e-8   2.06561e-8   1.88388e-8      9.56965e-10  8.65762e-10
 4.60941e-6   4.27275e-6   3.95968e-6      3.30304e-7   3.03644e-7
 ⋮                                      ⋱               
 8.65762e-10  9.56965e-10  1.05751e-9  

In [46]:
# Chapter 3: System of Equations
ΣW0 = W0

function φSYS(Ж, Ю, Б)
    X = Ж[1:NS]
    C1 = Ж[NS+1:2*NS]
    PX, PY = Ж[2*NS+1:2*NS+2]

    Πi = Б[1]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    # Conditional Expected Quantities
    SΩπ = φSΩπ(Ю)
    SΩ_X_sum = sum(SΩπ .* X', dims=2)
    SΩ_C1_sum = sum(SΩπ .* C1', dims=2)

    # Aggregate Demand
    XΣ = SΩ_X_sum
    YΣ = (ΣW0 .- PX .* XΣ .- SΩ_C1_sum) ./ PY

    # Learning
    Ω = φΞΩI(Ю, XΣ, YΣ)

    # FOC's
    C2_values = (W0 .- PX .* X' .- C1') ./ PY .+ X' .* Δ2
    FOCX_values = β .* sum(Ω .* exp.(-γ .* C2_values) .* (-PX ./ PY .+ Δ2), dims=1)
    FOCC1_values = exp.(-γ .* C1') + β .* sum(Ω .* exp.(-γ .* C2_values) .* (-1.0 ./ PY), dims=1)

    # Market Clearing
    MCX = XΣ[ΞΠi[Πi]] - (θXbar - θX)
    MCY = YΣ[ΞΠi[Πi]] - (θYbar - θY)

    return vcat(FOCX_values', FOCC1_values', MCX, MCY)
end

φSYS (generic function with 1 method)

In [49]:
# Chapter 4: Zero-Info Starting Point (ξ)
W0ξ = W0

# Fixed Point Zero info
function φSYSξ(Жξ,Б)
    PXξ, PYξ = Жξ[1:2]
    θX = ΞθX[Б[2]]
    θY = ΞθY[Б[3]]

    # Direct solution of MC's
    Xξ = (θXbar - θX)
    C1ξ = W0ξ - PXξ * (θXbar - θX) - PYξ *(θYbar - θY)

    # FOC's
    C2ξ_values = (W0ξ - PXξ * Xξ - C1ξ) / PYξ  .+ Xξ .* Δ2
    FOCXξ_values = β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-PXξ ./ PYξ .+ Δ2), dims=1)
    FOCC1ξ_values = exp.(-γ .* C1ξ) .+ β .* sum(πΩ .* exp.(-γ .* C2ξ_values) .* (-1.0 ./ PYξ), dims=1)

    return vcat(FOCXξ_values, FOCC1ξ_values)
end

# Starting Point of Full Code
function φЖ0(Б)
    θXi, θYi = Б[2], Б[3]
    θX = ΞθX[θXi]
    θY = ΞθY[θYi]
    solξ = nlsolve(Жξ -> φSYSξ(Жξ, [0, θXi, θYi]), [0.6, 0.7])
    PXξ, PYξ = solξ.zero
    Ж = zeros(2 * NS + 2)
    Ж[1:NS] .= θXbar - θX
    Ж[NS+1:2*NS] .= W0ξ .- PXξ * (θXbar .- θX) .- PYξ * (θYbar .- θY)
    Ж[2*NS+1:2*NS+2] .= [PXξ, PYξ]
    return Ж
end


φЖ0 (generic function with 1 method)

In [50]:
# Chapter 5: Solution over grid

sol_zeros = Array{Array{Float64}, 3}(undef, NΠ, NθX, NθY)

for Πi in 1:NΠ
    for θXi in 1:NθX
        for θYi in 1:NθY
            Ж_ini = φЖ0([Πi, θXi, θYi])
            for Ю_value in range(0.1, stop=1, length=7)
                print("\rProcessing: Πi = $Πi, θXi = $θXi, θYi = $θYi, Ю_value = $Ю_value")
                flush(stdout)
                sol = nlsolve(Ж -> φSYS(Ж, [Ю_value], [Πi, θXi, θYi]), Ж_ini)
                Ж_ini = sol.zero
            end
        end
    end
end


Processing: Πi = 5, θXi = 5, θYi = 5, Ю_value = 1.0



---

